# Task02 Subject05
Notebook to run the Brainplayback Task02 experiment for subject 05.

To recreate the environment run
`conda env create --name brainplayback --file environment.yml`

In [1]:
# Import requirements
import spotipy
import time
import pandas
import random
import numpy
import serial

In [2]:
# Import specific functions
from lib import *

In [3]:
# subject name/id
subject_id = "sub-05"

In [5]:
# Setup Spotipy
scope = "user-library-read,user-read-playback-state,user-modify-playback-state"
sp = spotipy.Spotify(
    auth_manager=spotipy.oauth2.SpotifyOAuth(
        scope=scope,
        client_id="253a1345850d4a5c94995c1159d286e4",
        client_secret="9a03e1bcb8124eed9b661991e84bf477",
        redirect_uri="http://127.0.0.1:9090",
    ),
)

# choose deviceID (1st)
res = sp.devices()
devID = res["devices"][0]["id"]

In [ ]:
# Setup COM Port - Trigger
# serialPort = open_trigger_port()

## Import subject-specific selection of songs

In [6]:
# read file
D = pandas.read_table(f"../data/music_matrices/music_{subject_id}.tsv", index_col="Emotion")

# extract list of emotions
emotionList = D.index.values

# Generate two new columns (start1MS and start2MS) in milliseconds for each song
for emotion in emotionList:
    D.loc[emotion, "startMS1"] = (int(D.start1[emotion][1:3]) * 60 + int(D.start1[emotion][4:6])) * 1000
    D.loc[emotion, "startMS2"] = (int(D.start2[emotion][1:3]) * 60 + int(D.start2[emotion][4:6])) * 1000

## Version 0.2a - 2 songs coupled by emotion

In [ ]:
run_id = "task-02a_run-1"

# create a new list with the same elements as emotionList, but in a random order
randomEmotionList = emotionList.copy()
random.shuffle(randomEmotionList)

# Initialize Dataframe to record timestamps
timeMatrix = pandas.DataFrame(
    columns=["emotion", "trackID", "timestamp0", "before_music_start", "after_music_start", "before_noise_start", "after_noise_start"]
)

# set initial volume
sp.volume(60, devID)

# wait for trigger
# wait_for_trigger(serialPort=serialPort, timeout=30)

# first baseline
timestamp0 = time.time()
sp.start_playback(uris=["spotify:track:04boE4u1AupbrGlI62WvoO"], position_ms=1000, device_id=devID)
wait_until(timestamp0 + 12)

# trial loop
for emotion in randomEmotionList:
    print(f"Now playing: {emotion}")

    for i in range(1, 3):
        song = D[f"trackID{str(i)}"][emotion]
        start_time = D[f"startMS{str(i)}"][emotion].item()

        print(f"Now playing: {song} starting at {str(start_time)}")

        timestamp1 = time.time()

        sp.start_playback(
            uris=[f"spotify:track:{song}"],
            position_ms=start_time,
            device_id=devID,
        )

        timestamp2 = time.time()

        fade_in(sp, devID, duration=2, steps=5, t1=timestamp1)

        wait_until(timestamp1 + 22)

        fade_out(sp, devID, duration=2, steps=5, t1=timestamp1 + 22)

        timestamp3 = time.time()

        sp.start_playback(
            uris=["spotify:track:04boE4u1AupbrGlI62WvoO"],
            position_ms=1000,
            device_id=devID,
        )

        timestamp4 = time.time()

        row = pandas.Series([emotion, song, timestamp0, timestamp1, timestamp2, timestamp3, timestamp4], index=timeMatrix.columns)
        timeMatrix = pandas.concat([timeMatrix, row.to_frame().T], ignore_index=True)

        wait_until(timestamp3 + 6)

    wait_until(timestamp3 + 6 + 12)

sp.pause_playback()

# save output

# theoretical onsets
theo_onset = numpy.array([12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

for i in range(1, 17, 2):
    theo_onset[i] = theo_onset[i - 1] + 30
    theo_onset[i + 1] = theo_onset[i] + 24 + 18
theo_onset[17] = theo_onset[16] + 30

# duplicate dataframe and calculate new colums
timeMatrix2 = timeMatrix
timeMatrix2["music_duration"] = timeMatrix["after_noise_start"] - timeMatrix["after_music_start"]
timeMatrix2["theo_music_onset"] = pandas.Series(theo_onset, name="theo_music_onset")
timeMatrix2["true_music_onset"] = timeMatrix["after_music_start"] - timeMatrix["timestamp0"]
timeMatrix2["diff_onset"] = timeMatrix2["true_music_onset"] - timeMatrix2["theo_music_onset"]
timeMatrix2["true_noise_onset"] = timeMatrix["after_noise_start"] - timeMatrix["timestamp0"]

# Save
timeMatrix2.to_csv(f"output_{subject_id}_{run_id}.tsv", sep="\t")
print("Done!")

KeyboardInterrupt: 

## Close COM Port

In [ ]:
# Close trigger port
# close_trigger_port(serialPort)